In [11]:
using JuMP, Ipopt, Plots, Printf, LinearAlgebra, SCS, Distributions

In [12]:
N = 3
λ = 1.0
ni = 4 * (N + 1)^2
np = 5 * N + 1
ns  = ni - 12
nt = np + ns + 1
println(ni)
println(np)
println(ns)
println(nt)


64
16
52
69


We Define the states now

In [13]:
S = Dict()
T = []
TG = []
TB = []
for ua in 0:N
    for ub in 0:N
        for sa in 0:1
            for sb in 0:1
                index = (2 * sa + sb) * (N + 1)^2 + (N + 1) * ua + ub
                S[index] = (ua, ub, sa, sb)
            end
        end
    end
end
Skeyer = Dict(value => key for (key, value) in S);


Define Terminal states

In [14]:
# Initialize lists
T = []
TG = []
TB = []

# Loop through the ranges
for ua in 0:N
    for ub in 0:N
        for sa in 0:1
            for sb in 0:1
                i = (2 * sa + sb) * (N + 1)^2 + (N + 1) * ua + ub
                if ua == N && ub == 0
                    push!(TG, i)
                    push!(T, i)
                end
                if ua == 0 && ub == N
                    push!(TG, i)
                    push!(T, i)
                end
                if ua == N && ub == N
                    push!(TB, i)
                    push!(T, i)
                end
                S[i] = (ua, ub, sa, sb)
            end
        end
    end
end
Tc = [i for i in 0:ni-1 if i ∉ T]

sort!(T);
sort!(TG);
sort!(TB);


println("Terminal States")
println("T: ", T)
println("TG: ", TG)
println("TB: ", TB)
println("Tc: ", Tc)
# println(size(Tc))

Terminal States
T: Any[3, 12, 15, 19, 28, 31, 35, 44, 47, 51, 60, 63]
TG: Any[3, 12, 19, 28, 35, 44, 51, 60]
TB: Any[15, 31, 47, 63]
Tc: [0, 1, 2, 4, 5, 6, 7, 8, 9, 10, 11, 13, 14, 16, 17, 18, 20, 21, 22, 23, 24, 25, 26, 27, 29, 30, 32, 33, 34, 36, 37, 38, 39, 40, 41, 42, 43, 45, 46, 48, 49, 50, 52, 53, 54, 55, 56, 57, 58, 59, 61, 62]


Parameters

In [15]:
# f⁺ = collect([[N * (j-1) + i for j in 1:2] for i in 1:N]);
# f⁺=reduce(hcat,f⁺)
# f⁻ = collect([[N * (j-1) + i + (2 * N) for j in 1:2] for i in 1:N]);
# f⁻=reduce(hcat,f⁻)
# g = [4 * N  + i for i in 1:N];
# koff = 5 * N + 1

f⁺=rand(N, 2)
f⁻=rand(N, 2)
f⁺_flattened = collect(Iterators.flatten(f⁺'))
f⁻_flattened = collect(Iterators.flatten(f⁻'))
g=rand(N)
koff=rand(Uniform(0,1))
P = collect(vcat(f⁺_flattened, f⁻_flattened, g, [koff]))
# println(P)


16-element Vector{Float64}:
 0.4438281417379347
 0.02407293122872345
 0.7097410300646111
 0.6214555457447279
 0.6472495112963288
 0.23696203042068087
 0.8598528163572163
 0.14296206804825817
 0.6035209908975165
 0.5569373222888053
 0.59584943113792
 0.6890496110846002
 0.9805757629706341
 0.1423438821098162
 0.056897703874896455
 0.8691256353996689

Defining matrices

In [16]:
M=zeros(Int, ni, ni, np);
for (u, u_) in Iterators.product(values(S), values(S))
    # Skip if u and u_ are the same
    if u == u_
        continue
    end

    i = Skeyer[u]
    j = Skeyer[u_]
    flag = 0
    ua, ub, sa, sb = u
    ua_, ub_, sa_, sb_ = u_

    tempk = 0

    if ua_==ua+1 && ub == ub_ && sa == sa_ && sb == sb_
        flag = 1
        tempk = N * sa + ua + 1
    elseif ua_==ua-1 && ub == ub_ && sa == sa_ && sb == sb_
        flag = 1
        tempk = sa * N + ua + 2 * N 
    elseif ub_==ub+1 && ua == ua_ && sa == sa_ && sb == sb_
        flag = 1
        tempk = N * sb + ub + 1
    elseif ub_==ub-1 && ua == ua_ && sa == sa_ && sb == sb_
        flag = 1
        tempk = sb * N + ub + 2 * N 
    elseif ua == ua_ && ub == ub_ && ub != 0 && sb == sb_ && sa==0 && sa_==1
        flag = 1
        tempk = 4 * N + ub 
    elseif ua == ua_ && ub == ub_ && sb == sb_ && sa==1 && sa_==0
        flag = 1
        tempk = 5 * N + 1
    elseif ua == ua_ && ub == ub_ && ua != 0 && sa == sa_ && sb==0 && sb_==1
        flag = 1
        tempk = 4 * N + ua 
    elseif ua == ua_ && ub == ub_ && sa == sa_ && sb==1 && sb_==0
        flag = 1
        tempk = 5 * N + 1
    end

    if flag == 1
        M[i+1, j+1, tempk] = flag
    end
end
for k in 1:np
    # Calculate the sum of each row for the kth slice of M (equivalent to M[:,:,k] in Python)
    M_i = [sum(M[:, :, k][i, :]) for i in 1:ni]

    for u in values(S)
        i = Skeyer[u]
        M[i+1, i+1, k] = -M_i[i+1]
    end
end

In [17]:
# P_=reshape(P, np, 1)
println(P)
Q = reduce((x, y) -> x + y, [P[k] * M[:, :, k] for k in 1:np])

[0.4438281417379347, 0.02407293122872345, 0.7097410300646111, 0.6214555457447279, 0.6472495112963288, 0.23696203042068087, 0.8598528163572163, 0.14296206804825817, 0.6035209908975165, 0.5569373222888053, 0.59584943113792, 0.6890496110846002, 0.9805757629706341, 0.1423438821098162, 0.056897703874896455, 0.8691256353996689]


64×64 Matrix{Float64}:
 -0.887656   0.443828   0.0        …   0.0        0.0        0.0
  0.859853  -2.30833    0.0240729      0.0        0.0        0.0
  0.0        0.142962  -1.43888        0.0        0.0        0.0
  0.0        0.0        0.603521       0.0        0.0        0.0
  0.859853   0.0        0.0            0.0        0.0        0.0
  0.0        0.859853   0.0        …   0.0        0.0        0.0
  0.0        0.0        0.859853       0.0        0.0        0.0
  0.0        0.0        0.0            0.0        0.0        0.0
  0.0        0.0        0.0            0.0        0.0        0.0
  0.0        0.0        0.0            0.0        0.0        0.0
  0.0        0.0        0.0        …   0.0        0.0        0.0
  0.0        0.0        0.0            0.0        0.0        0.0
  0.0        0.0        0.0            0.0        0.0        0.0
  ⋮                                ⋱                        
  0.0        0.0        0.0            0.0        0.0        0.0
  0.0 

In [18]:
function Qfinder(P,N::Int64,λ::Float64,S,Skeyer)
    ni = 4 * (N + 1)^2
    np = 5 * N + 1
    ns  = ni - 12
    nt = np + ns + 1
#     Q=@expression(model, zeros(AffExpr, ni, ni)) 
    Q=zeros(ni,ni)
    for (u, u_) in Iterators.product(values(S), values(S))
        if u == u_
            continue
        end
        i = Skeyer[u]
        j = Skeyer[u_]
        flag = 0
        ua, ub, sa, sb = u
        ua_, ub_, sa_, sb_ = u_

        tempk = 0

        if ua_==ua+1 && ub == ub_ && sa == sa_ && sb == sb_
            flag = 1
            tempk = N * sa + ua + 1
        elseif ua_==ua-1 && ub == ub_ && sa == sa_ && sb == sb_
            flag = 1
            tempk = sa * N + ua + 2 * N 
        elseif ub_==ub+1 && ua == ua_ && sa == sa_ && sb == sb_
            flag = 1
            tempk = N * sb + ub + 1
        elseif ub_==ub-1 && ua == ua_ && sa == sa_ && sb == sb_
            flag = 1
            tempk = sb * N + ub + 2 * N 
        elseif ua == ua_ && ub == ub_ && ub != 0 && sb == sb_ && sa==0 && sa_==1
            flag = 1
            tempk = 4 * N + ub 
        elseif ua == ua_ && ub == ub_ && sb == sb_ && sa==1 && sa_==0
            flag = 1
            tempk = 5 * N + 1
        elseif ua == ua_ && ub == ub_ && ua != 0 && sa == sa_ && sb==0 && sb_==1
            flag = 1
            tempk = 4 * N + ua 
        elseif ua == ua_ && ub == ub_ && sa == sa_ && sb==1 && sb_==0
            flag = 1
            tempk = 5 * N + 1
        end
        if flag == 1
            Q[i+1, j+1] = P[tempk]
        end
    end
    for i in 1:ni
        # Calculate the sum of each row for Qi
        qi = sum(Q[i, :])
        Q[i,i] = -qi
    end
    return Q
end

Q_=Qfinder(P,N,λ,S,Skeyer)
maximum(abs.(Q_-Q))

4.440892098500626e-16

In [21]:
R = zeros(ns, ni)
for i in 1:ns
    R[i,Tc[i]+1] = 1
end
println(size(Q))
println(size(R))

Qtilde = R * Q * R'
det(Qtilde)
# for rownum in 1:size(R,1)
#     println(R[rownum,:])
# end

(64, 64)
(52, 64)


2.220035748572265e19

In [22]:
Mtilde = zeros(ns, ns, np)
for k in 1:np
    Mtilde[:, :, k] = R * M[:, :, k] * R'
end
# Mtilde

In [257]:
# Mtilde = zeros(ns, ns, np)
# for i in 1:ns
#     for j in 1:ns
#         for k in 1:np
#             Mtilde[i, j, k] = sum(R[i, a] * R[j, b] * M[a, b, k] for a in 1:ni, b in 1:ni)
#         end
#     end
# end

In [258]:
Ai = zeros(ns, np, ns)
for i in 1:ns
    Ai[:,:,i]= Mtilde[i,:,:]   
end

In [259]:
# eis = Matrix{Float64}(I, ns, ns)
# # println(eis)
# Ai = zeros(ns, np, ns)
# for k in 1:ns
#     ei=eis[k,:]
#     for i in 1:ns
#         for j in 1:np
#             Ai[i,j,k]= ei'*Mtilde[:,i,j]   
#         end 
#     end
# end
# # minimum(Ai-Bi)

In [260]:
alpha=zeros(np,ns)
for i in 1:ns
    Si=Tc[i]
    for j in 1:ni
        for k in 1:np
            if (j-1 ∈ TB)
                alpha[k,i]+=M[Si+1,j,k]
            end
            
        end
    end
end

In [268]:
Di=zeros(nt,nt,ns)
for i in 1:ns
    Di[np+1:np+ns,1:np,i]=Ai[:,:,i]
    Di[1:np,nt,i]=λ*alpha[:,i]'
    Di[nt,nt,i]=1
end
for i in 1:ns
    println(det(Ai[:,:,i]))
end

LoadError: DimensionMismatch: matrix is not square: dimensions are (52, 16)

In [262]:
Ei=zeros(nt,nt,nt)
for i in 1:nt
    Ei[i,nt,i]=1
end
S0=Ei[:,:,np+1]

69×69 Matrix{Float64}:
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  

In [245]:
#Symmetrise the matrices
D_i=copy(Di)
for i in 1:ns
    D_i[ :, :,i] = (Di[ :, :,i] + Di[ :, :,i]') / 2
end
        
S_0 = (S0 + S0') / 2
E_i = copy(Ei)
for i in 1:nt
    E_i[ :, :,i] = (Ei[ :, :,i] + Ei[ :, :,i]') / 2
end

In [246]:
C_is=zeros(nt,nt,np)
for i in 1:np
    C_is[i,i,i]=1
end

We have defined all our matrices! We can move towards formulating a SDP problem for our case